### 框架干的最厉害的一件事就是帮我们把返向传播全部计算好了

In [17]:
import torch

需要求导的，可以手动定义：

In [18]:
#方法1
x = torch.randn(3,4,requires_grad=True)
x

In [19]:
#方法2
x = torch.randn(3,4)
x.requires_grad=True
x

In [20]:
b = torch.randn(3,4,requires_grad=True)

In [21]:
t = x + b

In [22]:
y = t.sum()
y

In [23]:
y.backward()

In [24]:
b.grad

虽然没有指定t的requires_grad但是需要用到它，也会默认的

In [25]:
x.requires_grad, b.requires_grad, t.requires_grad

### 举个例子看一下：

![title](img/2.png)

In [26]:
#计算流程
x = torch.rand(1)
b = torch.rand(1, requires_grad = True)
w = torch.rand(1, requires_grad = True)
y = w * x 
z = y + b 

In [27]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad#注意y也是需要的

In [28]:
x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf

返向传播计算

In [29]:
z.backward(retain_graph=True)#如果不清空会累加起来

In [30]:
w.grad

In [31]:
b.grad

### 做一个线性回归试试水

构造一组输入数据X和其对应的标签y

In [32]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

In [ ]:
y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

In [ ]:
import torch
import torch.nn as nn

### 线性回归模型
- 其实线性回归就是一个不加激活函数的全连接层

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

In [ ]:
model

指定好参数和损失函数

In [ ]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

训练模型

In [ ]:
for epoch in range(epochs):
    epoch += 1
    # 注意转行成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    # 梯度要清零每一次迭代
    optimizer.zero_grad() 

    # 前向传播
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)

    # 返向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

### 测试模型预测结果

In [ ]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

### 模型的保存与读取

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

In [ ]:
model.load_state_dict(torch.load('model.pkl'))

### 使用GPU进行训练
- 只需要把数据和模型传入到cuda里面就可以了

In [ ]:
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = nn.MSELoss()


learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 1000
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad() 

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))